# Bayesian Personalized Ranking
* See the corresponding file in `../../TrainingAlphas` for more details

In [1]:
const source_name = "MLE.Ensemble";

In [2]:
username = "Fro116"
task = "random"

"random"

In [3]:
import NBInclude: @nbinclude
import SparseArrays: sparse
@nbinclude("../Alpha.ipynb");

In [4]:
function get_query_features(alphas::Vector{String})
    A = Matrix{Float32}(undef, num_items(), length(alphas))
    @tprogress Threads.@threads for i = 1:length(alphas)
        A[:, i] = read_recommendee_alpha(alphas[i], "all").rating
    end
    collect(A')
end

function get_implicit_features()
    df = get_recommendee_split("implicit")
    sparse(df.item, df.user, df.rating, num_items(), 1)
end

function get_explicit_features()
    df = get_recommendee_split("explicit")
    sparse(df.item, df.user, df.rating, num_items(), 1)
end

function get_user_features()
    collect(vcat(get_implicit_features(), get_explicit_features()))
end

function get_embedding(
    u::Integer,
    a::Integer,
    q::Integer,
    user_features::AbstractMatrix,
    query_features::AbstractMatrix,
)
    user_features[:, u], [a], query_features[:, q]
end;

In [67]:
function get_embedding(
    u::Integer,
    a::Vector{Int32},
    user_features::AbstractMatrix,
    query_features::AbstractMatrix,
)
    repeat(user_features[:, u], 1, length(a)), a, query_features[:, a]
end;

In [109]:
function compute_alpha()
    chunk(arr, n) = [arr[i:min(i + n - 1, end)] for i = 1:n:length(arr)]
    scores = Array{Float32}(undef, num_items())
    batch_size = params["hyp"].batch_size
    batches = chunk(convert.(Int32, 1:num_items()), batch_size)
    @tprogress Threads.@threads for batch in batches
        sample = get_embedding(1, batch, U, Q)
        alpha = m(sample)
        scores[batch] .= vec(alpha)
    end
    write_recommendee_alpha(scores, "$task/$source_name")
end;

In [110]:
compute_alpha();